<h1>CEU Master Thesis</h1>
<br>
<font size="4">
    The Effects of Migration on Attitudes towards the European Union: Extent, Dynamics and Causality<br>
    by Alina Cherkas
</font>

_The notebook can be used to replicate the **country-level** dataset used in the master's thesis submitted to CEU in a.y. 2019/2020._

<h2>Table of Contents</h2>

- [Data Sources](#Data-Sources)
- [Preliminaries](#Preliminaries)
- [1. Data Cleaning](#1.-Data-Cleaning)
    - [1.1. Eurobarometer](#1.1.-Eurobarometer)
    - [1.2. Eurostat](#1.2.-Eurostat)
    - [1.3. OECD](#1.3.-OECD)
- [2. Shift-Share Instrument](#2.-Shift-Share-Instrument)
    - [2.1. Filtering Data](#2.1.-Filtering-Data)
    - [2.2. Calculating Share](#2.2.-Calculating-Share)
    - [2.3. Calculating Shift](#2.3.-Calculating-Shift)
- [3. Data Merging](#3.-Data-Merging)
- [Tables and Figures](#Tables-and-Figures)
    - [Table 1](#Table-1)
    - [Table 2](#Table-2)
    - [Table 3](#Table-3)
    - [Table 4](#Table-4)
    - [Table 5](#Table-5)
    - [Table 6](#Table-6)
    - [Table 7](#Table-7)
    - [Annex A](#Annex-A)
    - [Figure 1](#Figure-1)
    - [Figure 2](#Figure-2)
    - [Figure 3](#Figure-3)
    - [Figure 4](#Figure-4)
    
<h2>Output:</h2>

Running this script will create an output file `CEU_Thesis_Data.xlsx` with the following sheets:
- _Eurobarometer_ - data on attitudes
- _Eurostat_ - country-level control variables
- _OECDbyDestination_ - migration data per destination country and year
- _OECDbyOrigin_ - migration data per destination country, year and country of origin
- _OECDwithInstrument_ - same as _OECDbyDestination_ + calculated shift-share instrument
- _DatasetFull_ - the above sheets combined into one
- _DatasetFinal_ - a subset of _DatasetFull_ that is used for the actual analysis
- _Table 1. Descriptive Statistics_ - corresponds to Table 1 in the thesis.

## Data Sources

- [Eurobarometer](https://data.europa.eu/euodp/en/data/dataset?q=Eurobarometer&ext_boolean=all&sort=title_string_en+desc)
    - **Type:** Panel
    - **Level:** Aggregated Country-level Data
    - **Date Range:** 01/06/2005-29/11/2019
    - **Source:** [EU Data Portal](https://data.europa.eu/euodp/en/data/)
    - **Edition:** Standard Eurobarometer
    - **Volume:** A Countries
    - **Editions:** 63.4- 92.3
    - **File:** `Eurobarometer (Aggregated, 2005-2019).xlsx`

- [Eurostat](https://ec.europa.eu/eurostat/data/database)
    - **Type:** Panel
    - **Level:** Country-level Data
    - **Date Range:** 2008-2019
    - **Source:** [Eurostat](https://ec.europa.eu/eurostat/data/database)
    - **Indicators:**
        - TPS00001
        - TPS00003
        - TPS00010
        - TPS00028
        - TPS00203
        - TEC00001
        - TEC00114
        - TET00002
        - TEPSR_WC170
        - migr_imm1ctz
        - edat_lfse_03
    - **Files:**
        - `TPS000101589226555245.xlsx`
        - `TPS000101589226567287.xlsx`
        - `TPS000011589226035572.xlsx`
        - `TPS000031589225985326.xlsx`
        - `TPS000281589225952060.xlsx`
        - `TPS002031589226322086.xlsx`
        - `TEC000011591139254740.xlsx`
        - `TEC001141589226463773.xlsx`
        - `TET000021591139147531.xlsx`
        - `TET000021591139159026.xlsx`
        - `TEPSR_WC1701589226373441.xlsx`
        - `TEPSR_WC1701589226389035.xlsx`
        - `TEPSR_WC1701589226359818.xlsx`
        - `TEPSR_WC1701589226408745.xlsx`
        - `edat_lfse_03 migr_imm1ctz.xls`

- [OECD International Migration Database](https://stats.oecd.org/Index.aspx?DatasetCode=MIG#)
    - **Type:** Panel
    - **Level:** Country-level Data
    - **Date Range:** 2000-2018
    - **Source:** [OECD.Stat](https://ec.europa.eu/eurostat/data/database)
    - **Indicators:** Full Database Export
    - **File:** `MIG_24052020163240973.csv`

## Preliminaries

You need to install `altair` before running the code. Run `conda install -c conda-forge altair vega_datasets` in your console/terminal. See the [documentation](https://altair-viz.github.io/getting_started/installation.html) for details

In [1]:
# Standard library imports
import os, sys

# Third party imports
import numpy as np
import pandas as pd
import altair as alt

print('Loaded!')

Loaded!


In [2]:
# System information
print(f'Executable: {sys.executable}\nPython version: {sys.version}')
print(f'\nPackage verions:\n- Numpy: {np.__version__}\n- Pandas: {pd.__version__}\n- Altair: {alt.__version__}')

Executable: /Users/alinacherkas/opt/anaconda3/bin/python
Python version: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]

Package verions:
- Numpy: 1.18.1
- Pandas: 1.0.3
- Altair: 4.1.0


**Helper File**

In [3]:
# Name mappings and metadata
df_mapper = pd.read_excel('./Source Data/Auxiliary Data.xlsx', sheet_name = 'EU_Metadata')
print(f'Shape:{df_mapper.shape}')
display(df_mapper.head())

Shape:(28, 9)


,ISO,EC,Eurostat,Country,Country_OECD,YearJoined,Eurozone,Schengen,M49Standard
0,AT,AT,AT,Austria,Austria,1995,1,1,Western
1,BE,BE,BE,Belgium,Belgium,1958,1,1,Western
2,BG,BG,BG,Bulgaria,Czech Republic,2007,0,0,Eastern
3,HR,HR,HR,Croatia,NaN,2007,0,0,Southern
4,CY,CY,CY,Cyprus,NaN,2004,1,0,Western Asia


## 1. Data Cleaning

In this part, I read and clean data from various sources to prepare a country-level dataset for the main part of the analysis in the thesis.

In [4]:
# Creating an excel file to store thesis dataset
writer = pd.ExcelWriter('CEU_Thesis_Data.xlsx', engine='xlsxwriter')

### 1.1. Eurobarometer

In [5]:
# Multi-sheet file with Eurobarometer data
xls = pd.ExcelFile('./Source Data/Eurobarometer (Aggregated, 2005-2019).xlsx')
print(xls.sheet_names)

['63.4', '64.2', '65.2', '66.1', '67.2', '68.1', '69.2', '70.1', '71.3', '72.4', '73.4', '74.2', '75.3', '76.3', '77.3', '78.1', '79.3', '80.1', '81.4', '82.3', '83.3', '84.3', '85.2', '86.2', '87.3', '88.3', '89.1', '90.3', '91.5', '92.3']


In [6]:
df_list = []

# Looping through each sheet name
for sheet in xls.sheet_names:
    
    # Reading each sheet
    df_lambda = pd.read_excel('./Source Data/Eurobarometer (Aggregated, 2005-2019).xlsx', sheet_name = sheet)
    
    # Extracting Fieldwork date
    date = df_lambda.iloc[0,12]
    date = date.split(':', 1)[-1].split('-')[-1].strip()
    
    # Re-reading the file and skipping the first few rows 
    df_lambda = pd.read_excel('./Source Data/Eurobarometer (Aggregated, 2005-2019).xlsx', sheet_name = sheet, 
                              skiprows = 8)
    assert df_lambda.shape[0] == 17, 'Some rows are missing'
    
    # Dropping rows with percentages and keeping only raw counts
    df_lambda.drop(range(2,17,2), inplace = True)
    df_lambda.drop('Unnamed: 0', axis = 1, inplace = True)
    df_lambda.rename({'Unnamed: 1':'Category'}, axis = 1, inplace = True)
    
    # Calculating share of respondents in each category
    df_lambda.iloc[1:, 1:] = df_lambda.iloc[1:, 1:].divide(df_lambda.iloc[0, 1:]).multiply(100)
    df_lambda.insert(0, 'Eurobarometer', sheet)
    df_lambda.insert(1, 'Date', date)
    
    df_list.append(df_lambda)

# Joining all rounds
df_eurobar = pd.concat(df_list, ignore_index = True)
print(f'Shape:{df_eurobar.shape}')
display(df_eurobar.head())

Shape:(270, 45)


,Eurobarometer,Date,Category,UE25\nEU25,BE,CZ,DK,D-W,DE,D-E,...,NC,CY (tcc),UE27\nEU27,MK,IS,ME,RS,UE28\nEU28,AL,UE28-UK\nEU28-UK
0,63.4,06/2005,TOTAL,24801.000000,1000,1083,1051,1015,1520,505,...,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63.4,06/2005,Très positive,8.402887,6.4,4.80148,4.94767,7.19212,6.38158,3.16832,...,46.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.4,06/2005,Assez positive,38.381517,49.3,37.7655,29.5909,36.6502,36.0526,33.8614,...,21.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63.4,06/2005,Neutre,32.139833,31.7,35.7341,41.1037,34.3842,36.6447,45.9406,...,13.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63.4,06/2005,Assez négative,14.442966,9.9,16.8052,18.5538,17.8325,16.9737,13.2673,...,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Mapping of French row names to English
to_replace = {'Très positive':'VeryPositive',
              'Assez positive':'FairlyPositive',
              'Neutre':'Neutral',
              'Assez négative':'FairlyNegative',
              'Très négative':'VeryNegative',
              'NSP':'DK',
              "Total 'Positive'":'Positive',
              "Total 'Négative'":'Negative',
              'Négative':'Negative',
              'Négatif':'Negative',
              'Positif':'Positive'}

In [8]:
print(f'Shape before:{df_eurobar.shape}')

# Reshaping from wide to long format
df_eurobar = df_eurobar.melt(id_vars = ['Eurobarometer', 'Date', 'Category'],
                             var_name = 'Country',
                             value_name = 'Share')

# Converting data types and replacing row names
df_eurobar['Date'] = pd.to_datetime(df_eurobar['Date'])
df_eurobar['Share'] = df_eurobar['Share'].astype(float)
df_eurobar['Category'].replace(to_replace, inplace = True)

# Subsetting the countries of interest and pivoting the table
df_eurobar = df_eurobar.query('Country in @df_mapper.Eurostat and Category != "TOTAL"')
df_eurobar = df_eurobar.pivot_table(index = ['Eurobarometer', 'Date', 'Country'], columns = 'Category')

# Resetting index and organising column names
df_eurobar.reset_index(inplace = True)
df_eurobar.columns = [x[1] if x[1] != '' else x[0] for x in df_eurobar.columns]

assert df_eurobar.shape == (840, 11), 'Some rows of columns are missing'
print(f'Shape after:{df_eurobar.shape}')
display(df_eurobar.head())

Shape before:(270, 45)
Shape after:(840, 11)


,Eurobarometer,Date,Country,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,VeryNegative,VeryPositive
0,63.4,2005-06-01,AT,1.700000,25.600000,25.800000,32.700000,35.400000,30.100000,7.100000,4.300000
1,63.4,2005-06-01,BE,0.500000,9.900000,49.300000,12.100000,31.700000,55.700000,2.200000,6.400000
2,63.4,2005-06-01,BG,5.893910,7.269155,38.703340,9.332024,24.950884,59.823183,2.062868,21.119843
3,63.4,2005-06-01,CY,0.990099,13.069307,40.396040,15.247525,27.722772,56.039604,1.980198,15.643564
4,63.4,2005-06-01,CZ,0.923361,16.805171,37.765466,20.683287,35.734072,42.659280,3.970452,4.801477


In [9]:
print(f'Shape before:{df_eurobar.shape}')

# Creating a mapping from country names to ISO codes
to_replace = dict(zip(df_mapper['Eurostat'], df_mapper['ISO']))
df_eurobar.insert(1, 'ISO', df_eurobar['Country'].replace(to_replace))

# Creating a mapping from ISO codes to conventional country names
to_replace = dict(zip(df_mapper['Eurostat'], df_mapper['Country']))
df_eurobar['Country'].replace(to_replace, inplace = True)

print(f'Shape after:{df_eurobar.shape}')
display(df_eurobar.head())

Shape before:(840, 11)
Shape after:(840, 12)


,Eurobarometer,ISO,Date,Country,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,VeryNegative,VeryPositive
0,63.4,AT,2005-06-01,Austria,1.700000,25.600000,25.800000,32.700000,35.400000,30.100000,7.100000,4.300000
1,63.4,BE,2005-06-01,Belgium,0.500000,9.900000,49.300000,12.100000,31.700000,55.700000,2.200000,6.400000
2,63.4,BG,2005-06-01,Bulgaria,5.893910,7.269155,38.703340,9.332024,24.950884,59.823183,2.062868,21.119843
3,63.4,CY,2005-06-01,Cyprus,0.990099,13.069307,40.396040,15.247525,27.722772,56.039604,1.980198,15.643564
4,63.4,CZ,2005-06-01,Czech Republic,0.923361,16.805171,37.765466,20.683287,35.734072,42.659280,3.970452,4.801477


In [10]:
# There is a Eurobarometer round every 6 months, each with data on 28 countries
display(df_eurobar.groupby(['Eurobarometer', 'Date']).size().sort_index())

Eurobarometer  Date      
63.4           2005-06-01    28
64.2           2005-10-01    28
65.2           2006-04-01    28
66.1           2006-09-01    28
67.2           2007-05-01    28
68.1           2007-10-01    28
69.2           2008-04-01    28
70.1           2008-10-01    28
71.3           2009-06-01    28
72.4           2009-11-01    28
73.4           2010-05-01    28
74.2           2010-11-01    28
75.3           2011-05-25    28
76.3           2011-11-20    28
77.3           2012-05-27    28
78.1           2012-11-18    28
79.3           2013-05-26    28
80.1           2013-11-17    28
81.4           2014-06-14    28
82.3           2014-11-17    28
83.3           2015-05-27    28
84.3           2015-11-17    28
85.2           2016-05-30    28
86.2           2016-11-14    28
87.3           2017-05-30    28
88.3           2017-11-14    28
89.1           2018-03-28    28
90.3           2018-11-22    28
91.5           2019-01-07    28
92.3           2019-11-29    28
dtype: int64

In [11]:
df_eurobar.to_excel(writer, sheet_name='Eurobarometer', index = False)
print('Saved!')

Saved!


### 1.2. Eurostat

In [12]:
sheets = {'migr_imm1ctz1':'ImgrTotal',
          'migr_imm1ctz2':'ImgrEU28',
          'migr_imm1ctz3':'ImgrNonEU28',
          'edat_lfse_03':'EduShare'}

In [13]:
df_list = []

# Reading in education and migration data
for sheet, tag in sheets.items():
    
    df_lambda = pd.read_excel('./Source Data/Eurostat/edat_lfse_03 migr_imm1ctz.xls',
                              sheet_name = sheet, skiprows = 12, skipfooter = 3)
    df_lambda.rename({'GEO':'Eurostat', 'GEO(L)/TIME':'Country'}, axis = 1, inplace = True)
    df_lambda.insert(0, 'Measure', tag)
    
    df_list.append(df_lambda)
print('Completed!')

Completed!


In [14]:
files = {'TPS000011589226035572':'PopTotal',
         'TPS000031589225985326':'PopDensity',
         'TEC001141589226463773':'GDP_PPS',
         'TEC000011591139254740':'GDP',
         'TET000021591139159026':'Exports',
         'TET000021591139147531':'Imports',
         'TPS002031589226322086':'UnempRate',
         'TPS000101589226555245':'Pop15_24',
         'TPS000101589226567287':'Pop25_49',
         'TPS000281589225952060':'PopOver65',
         'TEPSR_WC1701589226359818':'UnempRate15_24',
         'TEPSR_WC1701589226373441':'UnempRate25_29',
         'TEPSR_WC1701589226408745':'UnempRate25_54',
         'TEPSR_WC1701589226389035':'UnempRate20_64'}

In [15]:
# Reading other Eurostat files one by one
for file, measure in files.items():
    
    df_lambda = pd.read_excel(f'./Source Data/Eurostat/{file}.xlsx', sheet_name = 'Sheet 1', skiprows = 10)
    
    # Dropping empty columns and footers
    df_lambda.drop([x for x in df_lambda.columns if 'Unnamed' in x], axis = 1, inplace = True)
    df_lambda.drop(0, inplace = True)
    df_lambda.drop(range(29, df_lambda.shape[0] + 1), inplace = True)
    
    # Renaming and inserting measure name
    df_lambda.rename({'TIME':'Eurostat', 'TIME.1':'Country'}, axis = 1, inplace = True)
    df_lambda.insert(0, 'Measure', measure)
    
    assert df_lambda.shape[0] == 28, 'Incorrect number of rows'
    df_list.append(df_lambda)
    
print('Completed!')

Completed!


In [16]:
# Joining all dataframes into one
df_eurostat = pd.concat(df_list)
print(f'Shape:{df_eurostat.shape}')
display(df_eurostat.head())

Shape:(504, 18)


,Measure,Eurostat,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2008,2007,2005,2006
0,ImgrTotal,BE,Belgium,:,135281,147377,129477,120078.0,123158.0,146626.0,123702.0,126703.0,137860.0,NaN,NaN,NaN,NaN,NaN
1,ImgrTotal,BG,Bulgaria,:,:,:,14103,18570.0,26615.0,25223.0,21241.0,25597.0,29559.0,NaN,NaN,NaN,NaN,NaN
2,ImgrTotal,CZ,Czechia,75620,48317,27114,34337,30124.0,29897.0,29602.0,64083.0,51847.0,65910.0,NaN,NaN,NaN,NaN,NaN
3,ImgrTotal,DK,Denmark,51800,52236,52833,54409,60312.0,68388.0,78492.0,74383.0,68579.0,64669.0,NaN,NaN,NaN,NaN,NaN
4,ImgrTotal,DE,Germany (until 1990 former territory of the FRG),346216,404055,489422,592175,692713.0,884893.0,1571047.0,1029852.0,917109.0,893886.0,NaN,NaN,NaN,NaN,NaN


In [17]:
# Names mapping as above
print(f'Shape before:{df_eurostat.shape}')

to_replace = dict(zip(df_mapper['Eurostat'], df_mapper['ISO']))
df_eurostat.insert(1, 'ISO', df_eurostat['Eurostat'].replace(to_replace))

to_replace = dict(zip(df_mapper['Eurostat'], df_mapper['Country']))
df_eurostat['Country'] = df_eurostat['Eurostat'].replace(to_replace)

df_eurostat.drop('Eurostat', axis = 1, inplace = True)
df_eurostat.replace({':':np.nan}, inplace = True)

print(f'Shape after:{df_eurostat.shape}')
display(df_eurostat.head())

Shape before:(504, 18)
Shape after:(504, 18)


,Measure,ISO,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2008,2007,2005,2006
0,ImgrTotal,BE,Belgium,NaN,135281.0,147377.0,129477.0,120078.0,123158.0,146626.0,123702.0,126703.0,137860.0,NaN,NaN,NaN,NaN,NaN
1,ImgrTotal,BG,Bulgaria,NaN,NaN,NaN,14103.0,18570.0,26615.0,25223.0,21241.0,25597.0,29559.0,NaN,NaN,NaN,NaN,NaN
2,ImgrTotal,CZ,Czech Republic,75620.0,48317.0,27114.0,34337.0,30124.0,29897.0,29602.0,64083.0,51847.0,65910.0,NaN,NaN,NaN,NaN,NaN
3,ImgrTotal,DK,Denmark,51800.0,52236.0,52833.0,54409.0,60312.0,68388.0,78492.0,74383.0,68579.0,64669.0,NaN,NaN,NaN,NaN,NaN
4,ImgrTotal,DE,Germany,346216.0,404055.0,489422.0,592175.0,692713.0,884893.0,1571047.0,1029852.0,917109.0,893886.0,NaN,NaN,NaN,NaN,NaN


In [18]:
df_eurostat.to_excel(writer, sheet_name='Eurostat', index = False)
print('Saved!')

Saved!


### 1.3. OECD

In [19]:
# OECD International Migration Database
df_oecd = pd.read_csv('./Source Data/OECD Migration/MIG_24052020163240973.csv', low_memory= False)
print(f'Shape:{df_oecd.shape}')
display(df_oecd.head())

Shape:(458192, 13)


,CO2,Country of birth/nationality,VAR,Variable,GEN,Gender,COU,Country,YEA,Year,Value,Flag Codes,Flags
0,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2000,2000,887.0,NaN,NaN
1,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2001,2001,456.0,NaN,NaN
2,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2002,2002,660.0,NaN,NaN
3,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2003,2003,1015.0,NaN,NaN
4,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2004,2004,1340.0,NaN,NaN


In [20]:
# Names mapping as above
print(f'Shape before:{df_oecd.shape}')

to_replace = dict(zip(df_mapper['Country_OECD'], df_mapper['ISO']))
df_oecd.insert(0, 'ISO', df_oecd['Country'].replace(to_replace))

to_replace = dict(zip(df_mapper['Country_OECD'], df_mapper['Country']))
df_oecd['Country'].replace(to_replace, inplace = True)

print(f'Shape after:{df_oecd.shape}')
display(df_oecd.head())

Shape before:(458192, 13)
Shape after:(458192, 14)


,ISO,CO2,Country of birth/nationality,VAR,Variable,GEN,Gender,COU,Country,YEA,Year,Value,Flag Codes,Flags
0,Australia,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2000,2000,887.0,NaN,NaN
1,Australia,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2001,2001,456.0,NaN,NaN
2,Australia,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2002,2002,660.0,NaN,NaN
3,Australia,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2003,2003,1015.0,NaN,NaN
4,Australia,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUS,Australia,2004,2004,1340.0,NaN,NaN


In [21]:
print('Countries missing in OECD data:', set(df_mapper['ISO']) - set(df_oecd['ISO']))
print('Countries missing in the Mapper data:', set(df_oecd['ISO']) - set(df_mapper['ISO']))

Countries missing in OECD data: {'MT', 'CZ', 'CY', 'HR', 'RO', 'LT'}
Countries missing in the Mapper data: {'New Zealand', 'United States', 'Turkey', 'Japan', 'Canada', 'Switzerland', 'Norway', 'Chile', 'Israel', 'Australia', 'Korea', 'Mexico', 'Iceland'}


In [22]:
# Variables of interest
to_keep = ['Inflows of foreign population by nationality', 'Stock of foreign population by nationality']

In [23]:
# Filtering data for only relevant countries and keeping only two variables
print(f'Shape before:{df_oecd.shape}')
df_oecd = df_oecd.query('Country in @df_mapper.Country and Variable in @to_keep')
print(f'Shape after:{df_oecd.shape}')
display(df_oecd.head())

Shape before:(458192, 14)
Shape after:(100721, 14)


,ISO,CO2,Country of birth/nationality,VAR,Variable,GEN,Gender,COU,Country,YEA,Year,Value,Flag Codes,Flags
53,AT,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2000,2000,828.0,NaN,NaN
54,AT,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2001,2001,1539.0,NaN,NaN
55,AT,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2002,2002,1101.0,NaN,NaN
56,AT,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2003,2003,743.0,NaN,NaN
57,AT,AFG,Afghanistan,B11,Inflows of foreign population by nationality,TOT,Total,AUT,Austria,2004,2004,650.0,NaN,NaN


In [24]:
# Dropping and renaming columns
print(f'Shape before:{df_oecd.shape}')
df_oecd.drop(['CO2', 'VAR', 'GEN', 'Gender', 'COU', 'YEA', 'Flag Codes', 'Flags'], axis = 1, inplace = True)
df_oecd.rename({'Country of birth/nationality':'CountryOrigin'}, axis = 1, inplace = True)
print(f'Shape after:{df_oecd.shape}')
display(df_oecd.head())

Shape before:(100721, 14)
Shape after:(100721, 6)


,ISO,CountryOrigin,Variable,Country,Year,Value
53,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2000,828.0
54,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2001,1539.0
55,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2002,1101.0
56,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2003,743.0
57,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2004,650.0


In [25]:
display(df_oecd['Variable'].value_counts())

Inflows of foreign population by nationality    53805
Stock of foreign population by nationality      46916
Name: Variable, dtype: int64

**Extracting Total Migration Per Country of Destination**

In [26]:
# Selecting data on country-level migration
df_lambda = df_oecd.query('CountryOrigin == "Total"').copy()
print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape after:(751, 6)


,ISO,CountryOrigin,Variable,Country,Year,Value
205901,AT,Total,Inflows of foreign population by nationality,Austria,2000,65954.0
205902,AT,Total,Inflows of foreign population by nationality,Austria,2001,74786.0
205903,AT,Total,Inflows of foreign population by nationality,Austria,2002,86144.0
205904,AT,Total,Inflows of foreign population by nationality,Austria,2003,93341.0
205905,AT,Total,Inflows of foreign population by nationality,Austria,2004,104246.0


In [27]:
# Dropping a column, pivoting and renaming the columns
print(f'Shape before:{df_lambda.shape}')

df_lambda.drop('CountryOrigin', axis = 1, inplace = True)
df_lambda = df_lambda.pivot_table(index = ['ISO', 'Country', 'Year'], columns = 'Variable').reset_index()
df_lambda.columns = ['ISO', 'Country', 'Year', 'InflowsTotal', 'StockTotal']

print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape before:(751, 6)
Shape after:(408, 5)


,ISO,Country,Year,InflowsTotal,StockTotal
0,AT,Austria,2000,65954.0,693955.0
1,AT,Austria,2001,74786.0,701768.0
2,AT,Austria,2002,86144.0,730261.0
3,AT,Austria,2003,93341.0,746753.0
4,AT,Austria,2004,104246.0,754216.0


In [28]:
df_lambda.to_excel(writer, sheet_name = 'OECDbyDestination', index = False)
print('Saved!')

Saved!


**Extracting Migration Per Country of Origin**

In [29]:
to_remove = ['Former Czechoslovakia', 'Former USSR', 'Former Yugoslavia', 'Not stated', 'Stateless', 'Total',
             'Unknown']

In [30]:
# Removing some groupped Countries of Origin 
print(f'Shape before:{df_oecd.shape}')
df_lambda = df_oecd.query('CountryOrigin not in @to_remove')
print(f'Shape after:{df_lambda.shape}')
print('Unique Countries of Origin:', df_lambda['CountryOrigin'].nunique())
display(df_lambda.head())

Shape before:(100721, 6)
Shape after:(98173, 6)
Unique Countries of Origin: 203


,ISO,CountryOrigin,Variable,Country,Year,Value
53,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2000,828.0
54,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2001,1539.0
55,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2002,1101.0
56,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2003,743.0
57,AT,Afghanistan,Inflows of foreign population by nationality,Austria,2004,650.0


In [31]:
to_keep = ['ISO', 'Country', 'Year', 'CountryOrigin']

In [32]:
# Pivoting and renaming columns
print(f'Shape before:{df_lambda.shape}')

df_lambda = df_lambda.pivot_table(index = to_keep, columns = 'Variable').reset_index()
df_lambda.columns = ['ISO', 'Country', 'Year', 'CountryOrigin', 'Inflows', 'Stock']

print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape before:(98173, 6)
Shape after:(60344, 6)


,ISO,Country,Year,CountryOrigin,Inflows,Stock
0,AT,Austria,2000,Afghanistan,828.0,NaN
1,AT,Austria,2000,Albania,192.0,NaN
2,AT,Austria,2000,Algeria,80.0,NaN
3,AT,Austria,2000,Andorra,1.0,NaN
4,AT,Austria,2000,Angola,84.0,NaN


In [33]:
print('Missing values by column:')
display(df_lambda.isna().sum())
print('Unique values by column:')
display(df_lambda.nunique())

Missing values by column:


ISO                  0
Country              0
Year                 0
CountryOrigin        0
Inflows           7844
Stock            14697
dtype: int64

Unique values by column:


ISO                 22
Country             22
Year                19
CountryOrigin      203
Inflows           5007
Stock            11371
dtype: int64

In [34]:
df_lambda.to_excel(writer, sheet_name = 'OECDbyOrigin', index = False)
print('Saved!')

Saved!


In [35]:
writer.save()
print('Saved!')

Saved!


## 2. Shift-Share Instrument

In this section, I calculate shift-share instument, bulding on the formula provided in Jaeger et al. (2018). The original equation reads:

<br>
<br>

<font size="5">
    $\tilde{m_{jt}}=
    \Sigma{_o^N}
    \frac{M_{ojt_0}}{M_{ot_0}}
    \frac{\Delta M_{ot}}{L_{jt-1}}=
     \Sigma{_o^N}
    \lambda{_o}
    \Delta
    $
</font>

<br>
<br>

<div style="text-align: justify">
    
"*where ${M_{ojt^0}} / {M_{ot^0}}$ is the share of immigrants from country of origin $o$ in location $j$ at reference date $t^0$ that predates t, $\Delta M_{ot}$ is the number of new arrivals from that country at time $t$ at the national level, and $L_{jt-1}$ is the local population in the previous period. The expected inflow rate $\tilde{m_{jt}}$ is therefore a weighted average of the national inflow rates from each country of origin (the “shift”), with weights that depend on
the distribution of earlier immigrants at $t^0$ (the “shares”)*" (Jaeger et al, 2018, p.5).
</div>

***

I adopt the above equation to migration in European countries. Therefore, I calculate the predicted immigrant flow using the following quation:

<br>
<br>

<font size="5">
    $\tilde{ImFlow_{c,t}}=
    \Sigma{_o^N}
    \frac{M_{o,c,t_0}}{M_{o,t_0}}
    \frac{\Delta M_{o,t}}{L_{c,t-1}}=
     \Sigma{_o^N}
    \lambda{_o}
    \Delta
    $
</font>

<br>
<br>

Where  $\tilde{ImFlow_{c,t}}$ is the expected inflow in country $c$ at time $t$. The first fraction, $M_{o,c,t_0}/M_{o,t_0}$,
is the share of immigrants from country of origin $o$ in **destination country $c$** at some prior period $t_o$. The second fraction consists of the total number of new arrivals from that country of origin **in Europe** at time $t$, denoted by $\Delta M_{o,t}$, and the population in destination country $c$ in the previous period. Thus, the instrument is a weighted mean of the inflow rates in Europe from each source country that depends on the prior distribution of immigrants.

**Citation:**
Jaeger, David A., et al. Shift-Share Instruments and the Impact of Immigration. Working Paper, 24285, National Bureau of Economic Research, Feb. 2018. National Bureau of Economic Research, [doi:10.3386/w24285](https://www.doi.org/10.3386/w24285).
    


In [36]:
writer = pd.ExcelWriter('CEU_Thesis_Data.xlsx', engine='openpyxl', mode = 'a')

In [37]:
df_instument = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'OECDbyOrigin')
print(f'Shape:{df_instument.shape}')
display(df_instument.head())

Shape:(60344, 6)


,ISO,Country,Year,CountryOrigin,Inflows,Stock
0,AT,Austria,2000,Afghanistan,828.0,NaN
1,AT,Austria,2000,Albania,192.0,NaN
2,AT,Austria,2000,Algeria,80.0,NaN
3,AT,Austria,2000,Andorra,1.0,NaN
4,AT,Austria,2000,Angola,84.0,NaN


### 2.1. Filtering Data

In [38]:
# Selecting candidate periods for t_0
df_lambda = df_instument.query('Year < 2008').dropna(subset = ['Stock'])
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(14523, 6)


,ISO,Country,Year,CountryOrigin,Inflows,Stock
173,AT,Austria,2000,Turkey,7096.0,129631.0
363,AT,Austria,2001,Turkey,7764.0,127264.0
380,AT,Austria,2002,Afghanistan,1101.0,2065.0
381,AT,Austria,2002,Albania,226.0,1594.0
382,AT,Austria,2002,Algeria,109.0,313.0


In [39]:
# Calculating how many countries of origin are reported per country per year
df_lambda = df_lambda.groupby(['ISO', 'Country', 'Year'], as_index = False).agg({'CountryOrigin':'count'})
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(136, 4)


,ISO,Country,Year,CountryOrigin
0,AT,Austria,2000,1
1,AT,Austria,2001,1
2,AT,Austria,2002,190
3,AT,Austria,2003,190
4,AT,Austria,2004,190


In [40]:
# Subsetting countries for every year with information reported by at least 100 countries of origin
df_lambda = df_lambda.query('CountryOrigin > 100').groupby('Year').agg({'Country':['unique', 'nunique'],
                                                                        'CountryOrigin':'mean'})
print(f'Shape:{df_lambda.shape}')
display(df_lambda)

Shape:(8, 3)


Country         CountryOrigin
                                                 unique nunique          mean
Year                                                                         
2000  [Germany, Spain, Finland, Greece, Italy, Nethe...       7    180.571429
2001  [Germany, Spain, Finland, Greece, Italy, Nethe...       7    180.571429
2002  [Austria, Germany, Spain, Finland, Greece, Hun...       9    184.111111
2003  [Austria, Germany, Spain, Finland, Greece, Hun...       9    184.444444
2004  [Austria, Germany, Spain, Finland, Greece, Hun...       9    184.555556
2005  [Austria, Germany, Spain, Finland, Greece, Hun...       9    184.555556
2006  [Austria, Germany, Spain, Finland, Greece, Hun...      10    180.000000
2007  [Austria, Germany, Spain, Finland, Greece, Hun...      10    185.700000

In [41]:
# 2002 covers 184 countries of origin on average for 9 destinations
to_keep = df_lambda.loc[2002, ('Country', 'unique')].tolist()
print(to_keep)

['Austria', 'Germany', 'Spain', 'Finland', 'Greece', 'Hungary', 'Italy', 'Netherlands', 'Sweden']


In [42]:
# Subseting observations for the selected group of 9 countries and all periods
print(f'Shape before:{df_instument.shape}')
df_instument = df_instument.query('Country in @to_keep').copy()
print(f'Shape after:{df_instument.shape}')
display(df_instument.head())

Shape before:(60344, 6)
Shape after:(32029, 6)


,ISO,Country,Year,CountryOrigin,Inflows,Stock
0,AT,Austria,2000,Afghanistan,828.0,NaN
1,AT,Austria,2000,Albania,192.0,NaN
2,AT,Austria,2000,Algeria,80.0,NaN
3,AT,Austria,2000,Andorra,1.0,NaN
4,AT,Austria,2000,Angola,84.0,NaN


### 2.2. Calculating Share

<font size="5">
    $\lambda=\frac{M_{o,c,t_0}}{M_{o,t_0}}$ 
</font>

In [43]:
df_M = df_instument.query('Year == 2002')
print(f'Shape:{df_M.shape}')
display(df_M.head())

Shape:(1765, 6)


,ISO,Country,Year,CountryOrigin,Inflows,Stock
380,AT,Austria,2002,Afghanistan,1101.0,2065.0
381,AT,Austria,2002,Albania,226.0,1594.0
382,AT,Austria,2002,Algeria,109.0,313.0
383,AT,Austria,2002,Andorra,0.0,0.0
384,AT,Austria,2002,Angola,33.0,131.0


In [44]:
# M_o,t_0
df_lambda = df_M.groupby('CountryOrigin').agg(StockOrigin = ('Stock','sum')).reset_index()
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(203, 2)


,CountryOrigin,StockOrigin
0,Afghanistan,83715.0
1,Albania,383484.0
2,Algeria,60474.0
3,Andorra,895.0
4,Angola,12222.0


In [45]:
# Adding StockOrigin (M_o,t_0) back to the dataframe
print(f'Shape before:{df_M.shape}')
df_lambda = df_M.merge(df_lambda, on = 'CountryOrigin', how = 'left')
print(f'Shape after:{df_lambda.shape}')
display(df_lambda.sort_values(['CountryOrigin', 'Year', 'ISO']).head())

Shape before:(1765, 6)
Shape after:(1765, 7)


,ISO,Country,Year,CountryOrigin,Inflows,Stock,StockOrigin
0,AT,Austria,2002,Afghanistan,1101.0,2065.0,83715.0
190,DE,Germany,2002,Afghanistan,3896.0,71662.0,83715.0
393,ES,Spain,2002,Afghanistan,28.0,NaN,83715.0
593,FI,Finland,2002,Afghanistan,354.0,719.0,83715.0
796,GR,Greece,2002,Afghanistan,NaN,0.0,83715.0


In [46]:
# Calculating Lambda , i.e. stock share at t_o
# Adding .001 for smoothing and not multiplying by zero later on
df_lambda.eval('StockShareT0 = Stock / StockOrigin + 0.001', inplace = True)

In [47]:
print(f'Shape before:{df_M.shape}')
df_lambda.drop(['Year', 'Inflows', 'Stock', 'StockOrigin'], axis = 1, inplace = True)
print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape before:(1765, 6)
Shape after:(1765, 4)


,ISO,Country,CountryOrigin,StockShareT0
0,AT,Austria,Afghanistan,0.025667
1,AT,Austria,Albania,0.005157
2,AT,Austria,Algeria,0.006176
3,AT,Austria,Andorra,0.001000
4,AT,Austria,Angola,0.011718


### 2.3. Calculating Shift

<font size="5">
    $\Delta=\frac{\Delta M_{o,t}}{L_{c,t-1}}$
</font>

In [48]:
# Reading country-level data from Eurostat
df_L = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'Eurostat')
print(f'Shape:{df_L.shape}')
display(df_L.head())

Shape:(504, 18)


,Measure,ISO,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2008,2007,2005,2006
0,ImgrTotal,BE,Belgium,NaN,135281.0,147377.0,129477.0,120078.0,123158.0,146626.0,123702.0,126703.0,137860.0,NaN,NaN,NaN,NaN,NaN
1,ImgrTotal,BG,Bulgaria,NaN,NaN,NaN,14103.0,18570.0,26615.0,25223.0,21241.0,25597.0,29559.0,NaN,NaN,NaN,NaN,NaN
2,ImgrTotal,CZ,Czech Republic,75620.0,48317.0,27114.0,34337.0,30124.0,29897.0,29602.0,64083.0,51847.0,65910.0,NaN,NaN,NaN,NaN,NaN
3,ImgrTotal,DK,Denmark,51800.0,52236.0,52833.0,54409.0,60312.0,68388.0,78492.0,74383.0,68579.0,64669.0,NaN,NaN,NaN,NaN,NaN
4,ImgrTotal,DE,Germany,346216.0,404055.0,489422.0,592175.0,692713.0,884893.0,1571047.0,1029852.0,917109.0,893886.0,NaN,NaN,NaN,NaN,NaN


In [49]:
# Selecting population data, resharing, resetting index and sorting values
print(f'Shape before:{df_L.shape}')

df_L = df_L.query('Measure == "PopTotal"').drop(['Measure', 'Country'], axis = 1)
df_L = df_L.melt(id_vars = 'ISO', var_name = 'Year', value_name = 'Population')

df_L.dropna(inplace = True)
df_L.reset_index(drop = True, inplace = True)

df_L['Year'] = df_L['Year'].astype(int)
df_L.sort_values(['ISO', 'Year'], inplace = True)

print(f'Shape after:{df_L.shape}')
display(df_L.head())

Shape before:(504, 18)
Shape after:(336, 3)


,ISO,Year,Population
327,AT,2008,8307989.0
19,AT,2009,8335003.0
47,AT,2010,8351643.0
75,AT,2011,8375164.0
103,AT,2012,8408121.0


In [50]:
# Shifting the data to obtain L_j,t-1
df_L['Pop_tminus1'] = df_L.groupby(['ISO'])['Population'].transform(lambda s: s.shift(-1))
print(f'Shape:{df_L.shape}')
display(df_L.head())

Shape:(336, 4)


,ISO,Year,Population,Pop_tminus1
327,AT,2008,8307989.0,8335003.0
19,AT,2009,8335003.0,8351643.0
47,AT,2010,8351643.0,8375164.0
75,AT,2011,8375164.0,8408121.0
103,AT,2012,8408121.0,8451860.0


In [51]:
# Calculating total inflows to Europe per source country
df_deltaM = df_instument.groupby(['Year', 'CountryOrigin'], as_index = False).agg({'Inflows':'sum'})
print(f'Shape:{df_deltaM.shape}')
display(df_deltaM.head())

Shape:(3857, 3)


,Year,CountryOrigin,Inflows
0,2000,Afghanistan,9023.0
1,2000,Albania,33823.0
2,2000,Algeria,8357.0
3,2000,Andorra,68.0
4,2000,Angola,1213.0


In [52]:
# Adding L_c,t-1 to delta M
print(f'Shape before:{df_deltaM.shape}')
df_deltaM = df_deltaM.merge(df_L, on = 'Year')
print(f'Shape after:{df_deltaM.shape}')
display(df_deltaM.head())

Shape before:(3857, 3)
Shape after:(62524, 6)


,Year,CountryOrigin,Inflows,ISO,Population,Pop_tminus1
0,2008,Afghanistan,5541.0,AT,8307989.0,8335003.0
1,2008,Afghanistan,5541.0,BE,10666866.0,10753080.0
2,2008,Afghanistan,5541.0,BG,7518002.0,7467119.0
3,2008,Afghanistan,5541.0,CY,776333.0,796930.0
4,2008,Afghanistan,5541.0,CZ,10343422.0,10425783.0


In [53]:
# Calculating the shift, dropping redundant columns
print(f'Shape before:{df_deltaM.shape}')
df_deltaM.eval('Shift = Inflows / Pop_tminus1 * 100', inplace = True)
df_deltaM.drop(['Inflows', 'Population', 'Pop_tminus1'], axis = 1, inplace = True)
print(f'Shape after:{df_deltaM.shape}')
display(df_deltaM.head())

Shape before:(62524, 6)
Shape after:(62524, 4)


,Year,CountryOrigin,ISO,Shift
0,2008,Afghanistan,AT,0.066479
1,2008,Afghanistan,BE,0.051529
2,2008,Afghanistan,BG,0.074205
3,2008,Afghanistan,CY,0.695293
4,2008,Afghanistan,CZ,0.053147


In [54]:
# Adding the shift (delta) to share (lambda)
df_instrument = df_deltaM.merge(df_lambda, on = ['ISO', 'CountryOrigin'])
print(f'Shape:{df_instrument.shape}')
display(df_instrument.head())

Shape:(19415, 6)


,Year,CountryOrigin,ISO,Shift,Country,StockShareT0
0,2008,Afghanistan,AT,0.066479,Austria,0.025667
1,2009,Afghanistan,AT,0.110469,Austria,0.025667
2,2010,Afghanistan,AT,0.143233,Austria,0.025667
3,2011,Afghanistan,AT,0.204588,Austria,0.025667
4,2012,Afghanistan,AT,0.224897,Austria,0.025667


In [55]:
# Calculating shift-share per country of origin, year and per country of destination
df_instrument.eval('ShiftShare = StockShareT0 * Shift', inplace = True)

In [56]:
# Summing up shift-share across countries of origin, to obtain ImFlow_c,t
print(f'Shape before:{df_instrument.shape}')
df_instrument = df_instrument.groupby(['ISO', 'Country', 'Year'], as_index = False).agg({'ShiftShare':'sum'})
print(f'Shape after:{df_instrument.shape}')
display(df_instrument.head())

Shape before:(19415, 7)
Shape after:(99, 4)


,ISO,Country,Year,ShiftShare
0,AT,Austria,2008,1.009434
1,AT,Austria,2009,0.904602
2,AT,Austria,2010,0.956444
3,AT,Austria,2011,1.072163
4,AT,Austria,2012,1.135971


In [57]:
# Nine countries per year, 11 years
print('Available countries per year:')
display(df_instrument['Year'].value_counts())
print('Available years per country:')
display(df_instrument['Country'].value_counts().sort_index())

Available countries per year:


2018    9
2017    9
2016    9
2015    9
2014    9
2013    9
2012    9
2011    9
2010    9
2009    9
2008    9
Name: Year, dtype: int64

Available years per country:


Austria        11
Finland        11
Germany        11
Greece         11
Hungary        11
Italy          11
Netherlands    11
Spain          11
Sweden         11
Name: Country, dtype: int64

In [58]:
# Reading cleaned OECD migration data
df_oecd = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'OECDbyDestination')
print(f'Shape:{df_oecd.shape}')
display(df_oecd.head())

Shape:(408, 5)


,ISO,Country,Year,InflowsTotal,StockTotal
0,AT,Austria,2000,65954.0,693955.0
1,AT,Austria,2001,74786.0,701768.0
2,AT,Austria,2002,86144.0,730261.0
3,AT,Austria,2003,93341.0,746753.0
4,AT,Austria,2004,104246.0,754216.0


In [59]:
# Not all countries have migration data
display(df_oecd['Country'].value_counts().sort_index())

Austria           19
Belgium           19
Bulgaria          19
Denmark           19
Estonia           18
Finland           19
France            19
Germany           19
Greece            18
Hungary           19
Ireland           19
Italy             19
Latvia            19
Luxembourg        19
Netherlands       19
Poland            19
Portugal          19
Slovakia          19
Slovenia          11
Spain             19
Sweden            19
United Kingdom    19
Name: Country, dtype: int64

In [60]:
# Adding shift-share istrument to OECD data
print(f'Shape before:{df_oecd.shape}')
df_oecd = df_oecd.merge(df_instrument, on = ['ISO', 'Country', 'Year'], how = 'outer')
print(f'Shape after:{df_oecd.shape}')
display(df_oecd.head())

Shape before:(408, 5)
Shape after:(409, 6)


,ISO,Country,Year,InflowsTotal,StockTotal,ShiftShare
0,AT,Austria,2000,65954.0,693955.0,NaN
1,AT,Austria,2001,74786.0,701768.0,NaN
2,AT,Austria,2002,86144.0,730261.0,NaN
3,AT,Austria,2003,93341.0,746753.0,NaN
4,AT,Austria,2004,104246.0,754216.0,NaN


In [61]:
display(df_oecd.notna().sum())

ISO             409
Country         409
Year            409
InflowsTotal    372
StockTotal      379
ShiftShare       99
dtype: int64

In [62]:
df_oecd.to_excel(writer, sheet_name = 'OECDwithInstrument', index = False)
writer.save()
print('Saved!')

Saved!


## 3. Data Merging

In [63]:
writer = pd.ExcelWriter('CEU_Thesis_Data.xlsx', engine='openpyxl', mode = 'a')

### 3.1. Adding Metadata

In [64]:
df_alpha = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'Eurobarometer')
print(f'Shape:{df_alpha.shape}')
display(df_alpha.head())

Shape:(840, 12)


,Eurobarometer,ISO,Date,Country,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,VeryNegative,VeryPositive
0,63.4,AT,2005-06-01,Austria,1.700000,25.600000,25.800000,32.700000,35.400000,30.100000,7.100000,4.300000
1,63.4,BE,2005-06-01,Belgium,0.500000,9.900000,49.300000,12.100000,31.700000,55.700000,2.200000,6.400000
2,63.4,BG,2005-06-01,Bulgaria,5.893910,7.269155,38.703340,9.332024,24.950884,59.823183,2.062868,21.119843
3,63.4,CY,2005-06-01,Cyprus,0.990099,13.069307,40.396040,15.247525,27.722772,56.039604,1.980198,15.643564
4,63.4,CZ,2005-06-01,Czech Republic,0.923361,16.805171,37.765466,20.683287,35.734072,42.659280,3.970452,4.801477


In [65]:
df_alpha['Year'] = df_alpha['Date'].dt.year

In [66]:
# Averaging Eurobarometer rounds by year
print(f'Shape before:{df_alpha.shape}')
df_alpha = df_alpha.groupby(['ISO', 'Country', 'Year'], as_index = False).mean()
print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(840, 13)
Shape after:(420, 12)


,ISO,Country,Year,Eurobarometer,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,VeryNegative,VeryPositive
0,AT,Austria,2005,63.80,1.683333,26.574510,23.439216,34.340196,36.817647,27.108824,7.765686,3.669608
1,AT,Austria,2006,65.65,1.426488,24.298781,26.414874,29.364872,36.153694,33.104158,5.016878,6.689284
2,AT,Austria,2007,67.65,0.937130,23.791203,28.923321,29.318579,35.341685,34.402606,5.527376,5.479285
3,AT,Austria,2008,69.65,1.997757,27.561416,24.662463,33.004686,36.291825,28.605882,5.493270,3.993420
4,AT,Austria,2009,71.85,0.735329,21.757616,30.517959,27.139748,37.699077,34.377302,5.382132,3.859343


In [67]:
print(f'Shape before:{df_mapper.shape}')
df_mapper.drop(['EC', 'Eurostat', 'Country', 'Country_OECD'], axis = 1, inplace = True)
print(f'Shape after:{df_mapper.shape}')
display(df_mapper.head())

Shape before:(28, 9)
Shape after:(28, 5)


,ISO,YearJoined,Eurozone,Schengen,M49Standard
0,AT,1995,1,1,Western
1,BE,1958,1,1,Western
2,BG,2007,0,0,Eastern
3,HR,2007,0,0,Southern
4,CY,2004,1,0,Western Asia


In [68]:
# Merging the main dataset with metadata
print(f'Shape before:{df_alpha.shape}')
df_alpha = df_alpha.merge(df_mapper, on = 'ISO')
print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 12)
Shape after:(420, 16)


,ISO,Country,Year,Eurobarometer,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,VeryNegative,VeryPositive,YearJoined,Eurozone,Schengen,M49Standard
0,AT,Austria,2005,63.80,1.683333,26.574510,23.439216,34.340196,36.817647,27.108824,7.765686,3.669608,1995,1,1,Western
1,AT,Austria,2006,65.65,1.426488,24.298781,26.414874,29.364872,36.153694,33.104158,5.016878,6.689284,1995,1,1,Western
2,AT,Austria,2007,67.65,0.937130,23.791203,28.923321,29.318579,35.341685,34.402606,5.527376,5.479285,1995,1,1,Western
3,AT,Austria,2008,69.65,1.997757,27.561416,24.662463,33.004686,36.291825,28.605882,5.493270,3.993420,1995,1,1,Western
4,AT,Austria,2009,71.85,0.735329,21.757616,30.517959,27.139748,37.699077,34.377302,5.382132,3.859343,1995,1,1,Western


### 3.2. Adding Eurostat

In [69]:
df_lambda = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'Eurostat')
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(504, 18)


,Measure,ISO,Country,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2008,2007,2005,2006
0,ImgrTotal,BE,Belgium,NaN,135281.0,147377.0,129477.0,120078.0,123158.0,146626.0,123702.0,126703.0,137860.0,NaN,NaN,NaN,NaN,NaN
1,ImgrTotal,BG,Bulgaria,NaN,NaN,NaN,14103.0,18570.0,26615.0,25223.0,21241.0,25597.0,29559.0,NaN,NaN,NaN,NaN,NaN
2,ImgrTotal,CZ,Czech Republic,75620.0,48317.0,27114.0,34337.0,30124.0,29897.0,29602.0,64083.0,51847.0,65910.0,NaN,NaN,NaN,NaN,NaN
3,ImgrTotal,DK,Denmark,51800.0,52236.0,52833.0,54409.0,60312.0,68388.0,78492.0,74383.0,68579.0,64669.0,NaN,NaN,NaN,NaN,NaN
4,ImgrTotal,DE,Germany,346216.0,404055.0,489422.0,592175.0,692713.0,884893.0,1571047.0,1029852.0,917109.0,893886.0,NaN,NaN,NaN,NaN,NaN


In [70]:
df_lambda.drop('Country', axis = 1, inplace = True)

In [71]:
# Reshaping and changing data type
print(f'Shape before:{df_lambda.shape}')
df_lambda = df_lambda.melt(id_vars = ['ISO', 'Measure'], var_name = 'Year', value_name = 'Value')
df_lambda['Year'] = df_lambda['Year'].astype(int)
print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape before:(504, 17)
Shape after:(7560, 4)


,ISO,Measure,Year,Value
0,BE,ImgrTotal,2009,NaN
1,BG,ImgrTotal,2009,NaN
2,CZ,ImgrTotal,2009,75620.0
3,DK,ImgrTotal,2009,51800.0
4,DE,ImgrTotal,2009,346216.0


In [72]:
# Pivoting and changing index and column names
print(f'Shape before:{df_lambda.shape}')

df_lambda = df_lambda.pivot_table(index = ['ISO', 'Year'], columns = 'Measure')
df_lambda.reset_index(inplace = True)
df_lambda.columns = [x[1] if x[1] != '' else x[0] for x in df_lambda.columns]

print(f'Shape after:{df_lambda.shape}')
display(df_lambda.head())

Shape before:(7560, 4)
Shape after:(420, 20)


,ISO,Year,EduShare,Exports,GDP,GDP_PPS,ImgrEU28,ImgrNonEU28,ImgrTotal,Imports,Pop15_24,Pop25_49,PopDensity,PopOver65,PopTotal,UnempRate,UnempRate15_24,UnempRate20_64,UnempRate25_29,UnempRate25_54
0,AT,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5.2,6.5,4.8
1,AT,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,4.9,6.2,4.6
2,AT,2007,NaN,NaN,NaN,127.0,NaN,NaN,NaN,NaN,NaN,NaN,100.6,NaN,NaN,NaN,9.4,4.5,5.9,4.2
3,AT,2008,NaN,123258.8,293761.9,127.0,NaN,NaN,NaN,125301.4,12.3,37.4,100.9,17.1,8307989.0,2.7,8.5,3.8,5.1,3.6
4,AT,2009,NaN,98213.7,288044.0,128.0,NaN,NaN,69295.0,102569.1,12.3,37.2,101.2,17.4,8335003.0,3.5,10.7,5.0,7.0,4.7


In [73]:
# Merging the main dataset with Eurostat data
print(f'Shape before:{df_alpha.shape}')
df_alpha = df_alpha.merge(df_lambda, on = ['ISO', 'Year'])
print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 16)
Shape after:(420, 34)


,ISO,Country,Year,Eurobarometer,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,...,Pop15_24,Pop25_49,PopDensity,PopOver65,PopTotal,UnempRate,UnempRate15_24,UnempRate20_64,UnempRate25_29,UnempRate25_54
0,AT,Austria,2005,63.80,1.683333,26.574510,23.439216,34.340196,36.817647,27.108824,...,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5.2,6.5,4.8
1,AT,Austria,2006,65.65,1.426488,24.298781,26.414874,29.364872,36.153694,33.104158,...,NaN,NaN,NaN,NaN,NaN,NaN,9.8,4.9,6.2,4.6
2,AT,Austria,2007,67.65,0.937130,23.791203,28.923321,29.318579,35.341685,34.402606,...,NaN,NaN,100.6,NaN,NaN,NaN,9.4,4.5,5.9,4.2
3,AT,Austria,2008,69.65,1.997757,27.561416,24.662463,33.004686,36.291825,28.605882,...,12.3,37.4,100.9,17.1,8307989.0,2.7,8.5,3.8,5.1,3.6
4,AT,Austria,2009,71.85,0.735329,21.757616,30.517959,27.139748,37.699077,34.377302,...,12.3,37.2,101.2,17.4,8335003.0,3.5,10.7,5.0,7.0,4.7


### 3.3. Adding OECD with Instument

In [74]:
df_lambda = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'OECDwithInstrument')
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(409, 6)


,ISO,Country,Year,InflowsTotal,StockTotal,ShiftShare
0,AT,Austria,2000,65954.0,693955.0,NaN
1,AT,Austria,2001,74786.0,701768.0,NaN
2,AT,Austria,2002,86144.0,730261.0,NaN
3,AT,Austria,2003,93341.0,746753.0,NaN
4,AT,Austria,2004,104246.0,754216.0,NaN


In [75]:
# Merging the main dataset with OECD migration + instument data
print(f'Shape before:{df_alpha.shape}')
df_alpha = df_alpha.merge(df_lambda, on = ['ISO', 'Country', 'Year'], how = 'left')
print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 34)
Shape after:(420, 37)


,ISO,Country,Year,Eurobarometer,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,...,PopOver65,PopTotal,UnempRate,UnempRate15_24,UnempRate20_64,UnempRate25_29,UnempRate25_54,InflowsTotal,StockTotal,ShiftShare
0,AT,Austria,2005,63.80,1.683333,26.574510,23.439216,34.340196,36.817647,27.108824,...,NaN,NaN,NaN,11.0,5.2,6.5,4.8,97995.0,774401.0,NaN
1,AT,Austria,2006,65.65,1.426488,24.298781,26.414874,29.364872,36.153694,33.104158,...,NaN,NaN,NaN,9.8,4.9,6.2,4.6,82899.0,796666.0,NaN
2,AT,Austria,2007,67.65,0.937130,23.791203,28.923321,29.318579,35.341685,34.402606,...,NaN,NaN,NaN,9.4,4.5,5.9,4.2,91546.0,804779.0,NaN
3,AT,Austria,2008,69.65,1.997757,27.561416,24.662463,33.004686,36.291825,28.605882,...,17.1,8307989.0,2.7,8.5,3.8,5.1,3.6,94368.0,829679.0,1.009434
4,AT,Austria,2009,71.85,0.735329,21.757616,30.517959,27.139748,37.699077,34.377302,...,17.4,8335003.0,3.5,10.7,5.0,7.0,4.7,91660.0,860004.0,0.904602


### 3.4. Organising Data

In [76]:
# Calculating new variables
print(f'Shape before:{df_alpha.shape}')

df_alpha['GDP_Growth'] = df_alpha['GDP'].divide(df_alpha.groupby('ISO')['GDP'].shift(1))
df_alpha.eval('GDP_Growth = GDP_Growth * 100 - 100', inplace = True)
df_alpha.eval('GlobalExposure = (Imports + Exports) / GDP * 100', inplace = True)
df_alpha.eval('YearJoined = Year - YearJoined', inplace = True)
df_alpha.eval('FairlyNeutral = FairlyPositive + Neutral + FairlyNegative', inplace = True)
df_alpha.eval('Polarised = VeryPositive + VeryNegative', inplace = True)
df_alpha.eval('PNratio = Positive / Negative', inplace = True)

print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 37)
Shape after:(420, 42)


,ISO,Country,Year,Eurobarometer,DK,FairlyNegative,FairlyPositive,Negative,Neutral,Positive,...,UnempRate25_29,UnempRate25_54,InflowsTotal,StockTotal,ShiftShare,GDP_Growth,GlobalExposure,FairlyNeutral,Polarised,PNratio
0,AT,Austria,2005,63.80,1.683333,26.574510,23.439216,34.340196,36.817647,27.108824,...,6.5,4.8,97995.0,774401.0,NaN,NaN,NaN,86.831373,11.435294,0.789420
1,AT,Austria,2006,65.65,1.426488,24.298781,26.414874,29.364872,36.153694,33.104158,...,6.2,4.6,82899.0,796666.0,NaN,NaN,NaN,86.867350,11.706163,1.127339
2,AT,Austria,2007,67.65,0.937130,23.791203,28.923321,29.318579,35.341685,34.402606,...,5.9,4.2,91546.0,804779.0,NaN,NaN,NaN,88.056209,11.006661,1.173406
3,AT,Austria,2008,69.65,1.997757,27.561416,24.662463,33.004686,36.291825,28.605882,...,5.1,3.6,94368.0,829679.0,1.009434,NaN,84.612811,88.515703,9.486690,0.866722
4,AT,Austria,2009,71.85,0.735329,21.757616,30.517959,27.139748,37.699077,34.377302,...,7.0,4.7,91660.0,860004.0,0.904602,-1.94644,69.705601,89.974652,9.241475,1.266677


In [77]:
to_keep = ['InflowsTotal', 'StockTotal', 'ImgrEU28', 'ImgrNonEU28', 'ImgrTotal']

In [78]:
# Converting  variables to percentages and PopTotal to 10k
df_alpha.loc[:, to_keep] = df_alpha[to_keep].divide(df_alpha['PopTotal'], axis = 0).multiply(100)
df_alpha.eval('PopTotal = PopTotal / 100000', inplace = True)

In [79]:
# Difference between OECD and Eurostat data on inflows
display(df_alpha.eval('InflowsTotal - ImgrTotal').describe())

count    190.000000
mean      -0.099872
std        0.257632
min       -0.673247
25%       -0.230396
50%       -0.136733
75%       -0.025761
max        0.838719
dtype: float64

In [80]:
to_rename = {'ISO':'ISO',
             'Country':'Country',
             'Year':'Year',
             'Eurobarometer':'Eurobarometer',
             
             'StockTotal':'ImmigrantStock',
             'InflowsTotal':'InflowsOECD',    
             'ImgrTotal':'InflowsEurostat',
             'ImgrEU28':'InflowsEU28',
             'ImgrNonEU28':'InflowsNonEU28',
             'ShiftShare':'ShiftShare',
             
             'VeryPositive':'VeryPositive',
             'FairlyPositive':'FairlyPositive',
             'Neutral':'Neutral',
             'FairlyNegative':'FairlyNegative',
             'VeryNegative':'VeryNegative',
             'DK':'DK',
             'Positive':'Positive',    
             'FairlyNeutral':'FairlyNeutral',
             'Negative':'Negative',
             'Polarised':'Polarised',
             'PNratio':'PNratio',
             
             'GDP':'GDP',
             'GDP_PPS':'GDP_PPS',
             'GDP_Growth':'GDP_Growth',
             'GlobalExposure':'GlobalExposure',
             'Imports':'Imports',
             'Exports':'Exports',
             'UnempRate':'UnempRate',
             'UnempRate15_24':'UnempRate15_24',
             'UnempRate20_64':'UnempRate20_64',
             'UnempRate25_29':'UnempRate25_29',
             'UnempRate25_54':'UnempRate25_54',
             
             'EduShare':'EduShare',
             'Pop15_24':'Pop15_24',
             'Pop25_49':'Pop25_49',
             'PopTotal':'PopTotal',
             'PopDensity':'PopDensity',
             'PopOver65':'PopOver65',
             
             'YearJoined':'YearJoined',
             'Eurozone':'Eurozone',
             'Schengen':'Schengen',
             'M49Standard':'M49Standard'}

In [81]:
# Renaming and reordering columns
print(f'Shape before:{df_alpha.shape}')

df_alpha.rename(to_rename, axis = 1, inplace = True)
df_alpha = df_alpha.reindex(to_rename.values(), axis = 1)

print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 42)
Shape after:(420, 42)


,ISO,Country,Year,Eurobarometer,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare,...,EduShare,Pop15_24,Pop25_49,PopTotal,PopDensity,PopOver65,YearJoined,Eurozone,Schengen,M49Standard
0,AT,Austria,2005,63.80,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10,1,1,Western
1,AT,Austria,2006,65.65,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,1,1,Western
2,AT,Austria,2007,67.65,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.6,NaN,12,1,1,Western
3,AT,Austria,2008,69.65,9.986520,1.135871,NaN,NaN,NaN,1.009434,...,NaN,12.3,37.4,83.07989,100.9,17.1,13,1,1,Western
4,AT,Austria,2009,71.85,10.317981,1.099700,0.831373,NaN,NaN,0.904602,...,NaN,12.3,37.2,83.35003,101.2,17.4,14,1,1,Western


In [82]:
print('Missing values per column:')
display(df_alpha.isna().sum())

Missing values per column:


ISO                  0
Country              0
Year                 0
Eurobarometer        0
ImmigrantStock     186
InflowsOECD        204
InflowsEurostat    144
InflowsEU28        252
InflowsNonEU28     252
ShiftShare         321
VeryPositive         0
FairlyPositive       0
Neutral              0
FairlyNegative       0
VeryNegative         0
DK                   0
Positive             0
FairlyNeutral        0
Negative             0
Polarised            0
PNratio              0
GDP                 84
GDP_PPS             84
GDP_Growth         112
GlobalExposure      84
Imports             84
Exports             84
UnempRate           84
UnempRate15_24       0
UnempRate20_64       0
UnempRate25_29       0
UnempRate25_54       0
EduShare           140
Pop15_24            84
Pop25_49            84
PopTotal            84
PopDensity          84
PopOver65           84
YearJoined           0
Eurozone             0
Schengen             0
M49Standard          0
dtype: int64

In [83]:
df_alpha.to_excel(writer, sheet_name = 'DatasetFull', index = False)
print('Saved!')

Saved!


In [84]:
writer.save()
print('Saved!')

Saved!


### 3.5. Subsetting Data

In this section, I decide on which countries go into final dataset for analysis.

In [85]:
writer = pd.ExcelWriter('CEU_Thesis_Data.xlsx', engine='openpyxl', mode = 'a')

In [86]:
df_alpha = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'DatasetFull')
print(f'Shape:{df_alpha.shape}')
display(df_alpha.head())

Shape:(420, 42)


,ISO,Country,Year,Eurobarometer,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare,...,EduShare,Pop15_24,Pop25_49,PopTotal,PopDensity,PopOver65,YearJoined,Eurozone,Schengen,M49Standard
0,AT,Austria,2005,63.80,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10,1,1,Western
1,AT,Austria,2006,65.65,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,1,1,Western
2,AT,Austria,2007,67.65,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.6,NaN,12,1,1,Western
3,AT,Austria,2008,69.65,9.986520,1.135871,NaN,NaN,NaN,1.009434,...,NaN,12.3,37.4,83.07989,100.9,17.1,13,1,1,Western
4,AT,Austria,2009,71.85,10.317981,1.099700,0.831373,NaN,NaN,0.904602,...,NaN,12.3,37.2,83.35003,101.2,17.4,14,1,1,Western


In [87]:
to_keep = ['ImmigrantStock', 'InflowsOECD', 'InflowsEurostat', 'InflowsEU28', 'InflowsNonEU28', 'ShiftShare']

In [88]:
# A few countries have no data on stock at all
df_lambda = df_alpha.groupby('Country', as_index = False)[to_keep].agg(lambda s: s.isna().sum())
df_lambda

,Country,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare
0,Austria,4.0,5.0,5.0,9.0,9.0,4.0
1,Belgium,4.0,5.0,6.0,9.0,9.0,15.0
2,Bulgaria,4.0,5.0,8.0,9.0,9.0,15.0
3,Croatia,15.0,15.0,5.0,9.0,9.0,15.0
4,Cyprus,15.0,15.0,5.0,9.0,9.0,15.0
5,Czech Republic,15.0,15.0,5.0,9.0,9.0,15.0
6,Denmark,4.0,5.0,5.0,9.0,9.0,15.0
7,Estonia,8.0,5.0,5.0,9.0,9.0,15.0
8,Finland,4.0,5.0,5.0,9.0,9.0,4.0
9,France,4.0,5.0,5.0,9.0,9.0,15.0


In [89]:
# Countries with lots of missing records on Stock
to_drop = df_lambda.query('ImmigrantStock == 15')['Country'].tolist()
print(to_drop)

['Croatia', 'Cyprus', 'Czech Republic', 'Lithuania', 'Malta', 'Romania']


In [90]:
# No data on immigration before 2008/2007
df_alpha.groupby('Year')[to_keep].agg(lambda s: s.isna().sum())

,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare
Year,,,,,,
2005,28.0,28.0,28.0,28.0,28.0,28.0
2006,28.0,28.0,28.0,28.0,28.0,28.0
2007,28.0,28.0,28.0,28.0,28.0,28.0
2008,8.0,6.0,28.0,28.0,28.0,19.0
2009,8.0,6.0,2.0,28.0,28.0,19.0
2010,7.0,6.0,1.0,28.0,28.0,19.0
2011,7.0,6.0,1.0,28.0,28.0,19.0
2012,6.0,6.0,0.0,28.0,28.0,19.0
2013,6.0,7.0,0.0,0.0,0.0,19.0


In [91]:
# Subsetting observation for years 2009-2017 and removing countries with missing records on stock
print(f'Shape before:{df_alpha.shape}')
df_alpha = df_alpha.query('2008 < Year < 2018 and Country not in @to_drop').copy()
print(f'Shape after:{df_alpha.shape}')
display(df_alpha.head())

Shape before:(420, 42)
Shape after:(198, 42)


,ISO,Country,Year,Eurobarometer,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare,...,EduShare,Pop15_24,Pop25_49,PopTotal,PopDensity,PopOver65,YearJoined,Eurozone,Schengen,M49Standard
4,AT,Austria,2009,71.85,10.317981,1.099700,0.831373,NaN,NaN,0.904602,...,NaN,12.3,37.2,83.35003,101.2,17.4,14,1,1,Western
5,AT,Austria,2010,73.80,10.579703,1.160203,0.849869,NaN,NaN,0.956444,...,16.2,12.2,36.9,83.51643,101.5,17.6,15,1,1,Western
6,AT,Austria,2011,75.80,10.903703,1.312464,0.981832,NaN,NaN,1.072163,...,16.3,12.2,36.5,83.75164,101.8,17.6,16,1,1,Western
7,AT,Austria,2012,77.70,11.315596,1.493853,1.088912,NaN,NaN,1.135971,...,16.9,12.1,36.2,84.08121,102.3,17.8,17,1,1,Western
8,AT,Austria,2013,79.70,11.882213,1.599979,1.205249,0.712494,0.381466,1.209287,...,17.7,12.1,35.8,84.51860,102.9,18.1,18,1,1,Western


In [92]:
# Most countries have full records
df_alpha.groupby('Country')[to_keep].agg(lambda s: s.isna().sum())

,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare
Country,,,,,,
Austria,0.0,0.0,0.0,4.0,4.0,0.0
Belgium,0.0,0.0,1.0,4.0,4.0,9.0
Bulgaria,0.0,0.0,3.0,4.0,4.0,9.0
Denmark,0.0,0.0,0.0,4.0,4.0,9.0
Estonia,3.0,0.0,0.0,4.0,4.0,9.0
Finland,0.0,0.0,0.0,4.0,4.0,0.0
France,0.0,0.0,0.0,4.0,4.0,9.0
Germany,0.0,0.0,0.0,4.0,4.0,0.0
Greece,0.0,5.0,0.0,4.0,4.0,0.0


In [93]:
# EU - NonEU distinction is available only after 20012
df_alpha.groupby('Year')[to_keep].agg(lambda s: s.isna().sum())

,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare
Year,,,,,,
2009,2.0,0.0,2.0,22.0,22.0,13.0
2010,1.0,0.0,1.0,22.0,22.0,13.0
2011,1.0,0.0,1.0,22.0,22.0,13.0
2012,0.0,0.0,0.0,22.0,22.0,13.0
2013,0.0,1.0,0.0,0.0,0.0,13.0
2014,0.0,1.0,0.0,0.0,0.0,13.0
2015,0.0,1.0,0.0,0.0,0.0,13.0
2016,0.0,1.0,0.0,0.0,0.0,13.0
2017,0.0,1.0,0.0,0.0,0.0,13.0


In [94]:
# Creating IV Sample indicator, removing GR and HU from the sample (see the thesis)
df_alpha['InflowsOECD'] = df_alpha['InflowsOECD'].combine_first(df_alpha['InflowsEurostat'])

df_alpha['IV_Sample'] = df_alpha['ShiftShare'].notna()
df_alpha.loc[df_alpha['ISO'].isin(['HU', 'GR']), 'IV_Sample'] = False

df_alpha['EUnonEU_Sample'] = df_alpha['InflowsEU28'].notna()
df_alpha.reset_index(drop = True, inplace = True)

display(df_alpha[to_keep].describe().round(2))

,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare
count,194.00,198.00,194.00,110.00,110.00,81.00
mean,9.07,0.79,0.89,0.37,0.37,0.77
std,8.94,0.77,0.77,0.59,0.26,0.35
min,0.16,0.04,0.09,0.01,0.01,0.34
25%,4.68,0.34,0.49,0.10,0.18,0.50
50%,6.93,0.60,0.69,0.17,0.35,0.67
75%,11.16,0.96,1.10,0.43,0.47,0.95
max,47.66,4.02,4.23,3.01,1.23,1.78


In [95]:
df_alpha.to_excel(writer, sheet_name = 'DatasetFinal', index = False)
print('Saved!')

Saved!


In [96]:
writer.save()
writer.close()
print('Saved!')

Saved!


## Tables and Figures

In [97]:
writer = pd.ExcelWriter('CEU_Thesis_Data.xlsx', engine='openpyxl', mode = 'a')

In [98]:
df_alpha = pd.read_excel('CEU_Thesis_Data.xlsx', sheet_name = 'DatasetFinal')
print(f'Shape:{df_alpha.shape}')
display(df_alpha.head())

Shape:(198, 44)


,ISO,Country,Year,Eurobarometer,ImmigrantStock,InflowsOECD,InflowsEurostat,InflowsEU28,InflowsNonEU28,ShiftShare,...,Pop25_49,PopTotal,PopDensity,PopOver65,YearJoined,Eurozone,Schengen,M49Standard,IV_Sample,EUnonEU_Sample
0,AT,Austria,2009,71.85,10.317981,1.099700,0.831373,NaN,NaN,0.904602,...,37.2,83.35003,101.2,17.4,14,1,1,Western,True,False
1,AT,Austria,2010,73.80,10.579703,1.160203,0.849869,NaN,NaN,0.956444,...,36.9,83.51643,101.5,17.6,15,1,1,Western,True,False
2,AT,Austria,2011,75.80,10.903703,1.312464,0.981832,NaN,NaN,1.072163,...,36.5,83.75164,101.8,17.6,16,1,1,Western,True,False
3,AT,Austria,2012,77.70,11.315596,1.493853,1.088912,NaN,NaN,1.135971,...,36.2,84.08121,102.3,17.8,17,1,1,Western,True,False
4,AT,Austria,2013,79.70,11.882213,1.599979,1.205249,0.712494,0.381466,1.209287,...,35.8,84.51860,102.9,18.1,18,1,1,Western,True,True


### Table 1

In [99]:
to_keep = ['ISO', 'Country', 'Year', 'IV_Sample', 'ImmigrantStock', 'InflowsOECD', 'InflowsEurostat', 'ShiftShare',
           'VeryNegative', 'FairlyNegative', 'Negative', 'Positive', 'UnempRate', 'GDP_Growth',
           'GlobalExposure', 'Pop25_49', 'PopOver65']
to_drop = ['ISO', 'Country', 'Year', 'IV_Sample']

In [100]:
df_lambda = df_alpha[to_keep].copy()
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(198, 17)


,ISO,Country,Year,IV_Sample,ImmigrantStock,InflowsOECD,InflowsEurostat,ShiftShare,VeryNegative,FairlyNegative,Negative,Positive,UnempRate,GDP_Growth,GlobalExposure,Pop25_49,PopOver65
0,AT,Austria,2009,True,10.317981,1.099700,0.831373,0.904602,5.382132,21.757616,27.139748,34.377302,3.5,-1.946440,69.705601,37.2,17.4
1,AT,Austria,2010,True,10.579703,1.160203,0.849869,0.956444,7.750000,24.900000,32.650000,29.850000,3.2,2.726181,79.427070,36.9,17.6
2,AT,Austria,2011,True,10.903703,1.312464,0.981832,1.072163,9.226481,26.980142,36.206622,28.054755,3.0,4.809822,85.440303,36.5,17.6
3,AT,Austria,2012,True,11.315596,1.493853,1.088912,1.135971,8.962739,26.082578,35.045317,24.471299,3.2,2.748633,84.298877,36.2,17.8
4,AT,Austria,2013,True,11.882213,1.599979,1.205249,1.209287,7.781473,26.388307,34.169780,25.514613,3.6,1.649820,83.320748,35.8,18.1


In [101]:
to_columns = [('Full Sample', 'Mean'),
              ('Full Sample', 'Std. Dev.'),
              ('Full Sample', 'Range'),
              ('Full Sample less LU', 'Mean'),
              ('Full Sample less LU', 'Std. Dev.'),
              ('Full Sample less LU', 'Range'),
              ('IV Sample', 'Mean'),
              ('IV Sample', 'Std. Dev.'),
              ('IV Sample', 'Range')]

In [102]:
def get_range(Series):
    Min = Series.round(2).min()
    Max = Series.round(2).max()
    return '-'.join([str(Min), str(Max)])

In [103]:
df_lambda = pd.concat([df_lambda.drop(to_drop, axis = 1).mean().round(2),
                       df_lambda.drop(to_drop, axis = 1).std().round(2),
                       df_lambda.drop(to_drop, axis = 1).apply(get_range),
                       df_lambda.query('ISO != "LU"').drop(to_drop, axis = 1).mean().round(2),
                       df_lambda.query('ISO != "LU"').drop(to_drop, axis = 1).std().round(2),
                       df_lambda.query('ISO != "LU"').drop(to_drop, axis = 1).apply(get_range),
                       df_lambda.query('IV_Sample == True').drop(to_drop, axis = 1).mean().round(2),
                       df_lambda.query('IV_Sample == True').drop(to_drop, axis = 1).std().round(2),
                       df_lambda.query('IV_Sample == True').drop(to_drop, axis = 1).apply(get_range)],
                      axis = 1)
df_lambda.columns = pd.MultiIndex.from_tuples(to_columns, names=['Sample', 'Measure'])
print(f'Shape:{df_lambda.shape}')
print('Table 1. Descriptive Statistics')
display(df_lambda.head())

Shape:(13, 9)
Table 1. Descriptive Statistics


Sample          Full Sample                       Full Sample less LU  \
Measure                Mean Std. Dev.       Range                Mean   
ImmigrantStock         9.07      8.94  0.16-47.66                7.33   
InflowsOECD            0.79      0.77   0.04-4.02                0.65   
InflowsEurostat        0.89      0.77   0.09-4.23                0.74   
ShiftShare             0.77      0.35   0.34-1.78                0.77   
VeryNegative           4.68      3.59    0.6-19.4                4.75   

Sample                                IV Sample                       
Measure         Std. Dev.       Range      Mean Std. Dev.      Range  
ImmigrantStock       4.25  0.16-17.69      7.89      3.08  2.69-15.3  
InflowsOECD          0.45   0.04-2.48      0.93      0.50  0.34-2.48  
InflowsEurostat      0.41   0.09-1.94      0.89      0.35  0.42-1.94  
ShiftShare           0.35   0.34-1.78      0.81      0.38  0.34-1.78  
VeryNegative         3.65    0.6-19.4      4.55      2.23  0.89-9.95

In [104]:
df_lambda.to_excel(writer, sheet_name = 'Table 1. Descriptive Statistics')
print('Saved!')

Saved!


In [105]:
writer.save()
writer.close()
print('Saved!')

Saved!


### Table 2

Replication can be found in `2. Analysis.Rmd`

### Table 3

Replication can be found in `2. Analysis.Rmd`

### Table 4

Replication can be found in `2. Analysis.Rmd`

### Table 5

Replication can be found in `2. Analysis.Rmd`

### Table 6

Replication code can be found in `2. Analysis.Rmd`, but no data for replication is available due to sharing restrictions.

### Table 7

Replication code can be found in `2. Analysis.Rmd`, but no data for replication is available due to sharing restrictions.

### Annex A

Replication can be found in `2. Analysis.Rmd`

### Figure 1

The figure was created using _Tableau Desktop (2020.2)_ based on `DatasetFinal` from `CEU_Thesis_Data.xlsx`. Replication codes are not applicable.

### Figure 2

The figure was created using _Tableau Desktop (2020.2)_ based on `DatasetFinal` from `CEU_Thesis_Data.xlsx`. Replication codes are not applicable.

### Figure 3

Replication can be found in `2. Analysis.Rmd`

### Figure 4

In [106]:
to_keep = ['Country', 'Year', 'InflowsOECD', 'InflowsEurostat', 'ShiftShare']

In [107]:
df_lambda = df_alpha.query('IV_Sample == True')[to_keep]
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(63, 5)


,Country,Year,InflowsOECD,InflowsEurostat,ShiftShare
0,Austria,2009,1.099700,0.831373,0.904602
1,Austria,2010,1.160203,0.849869,0.956444
2,Austria,2011,1.312464,0.981832,1.072163
3,Austria,2012,1.493853,1.088912,1.135971
4,Austria,2013,1.599979,1.205249,1.209287


In [108]:
df_lambda = df_lambda.melt(id_vars = ['Country', 'Year'], var_name = 'Variable', value_name = 'Value')
assert df_lambda['Country'].nunique() == 7, 'Missing Countries from the IV Sample'
print(f'Shape:{df_lambda.shape}')
display(df_lambda.head())

Shape:(189, 4)


,Country,Year,Variable,Value
0,Austria,2009,InflowsOECD,1.099700
1,Austria,2010,InflowsOECD,1.160203
2,Austria,2011,InflowsOECD,1.312464
3,Austria,2012,InflowsOECD,1.493853
4,Austria,2013,InflowsOECD,1.599979


In [109]:
df_lambda['Variable'].replace({'InflowsOECD':'Immigant Flow (OECD)',
                               'InflowsEurostat':'Immigant Flow (Eurostat)',
                               'ShiftShare':'Shift-Share Instument'}, inplace = True)
display(df_lambda.nunique())

Country       7
Year          9
Variable      3
Value       189
dtype: int64

In [110]:
chart = alt.Chart(df_lambda).mark_line().encode(
    x = 'Year:N',
    y = alt.Y('Value:Q', title = 'Share (in %)'),
    color = alt.Color('Variable:O', legend=alt.Legend(orient = 'bottom'), scale=alt.Scale(scheme='set1'))
).properties(
    width = 200,
    height = 150
).facet(
    'Country',
    columns = 4
).configure_axisX(
    labelAngle = -45,
    grid = True
).configure_axis(
    labelFontSize = 8,
    titleFontSize = 12
).configure_header(
    titleFontSize=12,
    labelFontSize=12
).properties(
    title='Figure 4. Migrant Inflows and Shift-Share Instrument for 7 European Countries (2009-2017)'
).configure_title(
    fontSize=16,
    anchor='middle')

#chart.save('Figure 4.html')
chart

alt.FacetChart(...)

## End of the Script